In [ ]:
import random
import simpy
import os
import numpy as np
import pandas as pd
from mesa import Agent, Model
from mesa.time import RandomActivation
from pathlib import Path

# SimPy Queue System


Could not import SolaraViz. If you need it, install with 'pip install --pre mesa[viz]'


In [ ]:
# SimPy Queue System with Dynamic Capacity
# class Ride:
#     def __init__(self, env, capacity, service_time, popularity):
#         self.env = env
#         self.capacity = capacity
#         self.queue = simpy.Resource(env, capacity=self.capacity)
#         self.service_time = service_time
#         self.wait_times = []
#         self.popularity = popularity
#         self.queue_lengths = []  # Store queue lengths over time

#     def ride_experience(self, guest):
#         with self.queue.request() as request:
#             queue_length = len(self.queue.queue)
#             self.queue_lengths.append(queue_length)  # Track queue length

#             yield request  # Wait for turn
#             wait_time = self.env.now - guest.arrival_time
#             self.wait_times.append(wait_time)

#             yield self.env.timeout(self.service_time)  # Ride duration
#             print(f"Guest {guest.unique_id} waited {wait_time:.2f} mins and finished at {self.env.now:.2f}")

#     # def adjust_capacity(self):
#     #     """Dynamically adjust ride capacity if demand is high"""
#     #     if len(self.queue.queue) > 5:  # Increase capacity if long queue
#     #         self.capacity += 1
#     #         self.queue = simpy.Resource(self.env, capacity=self.capacity)
#     #         print(f"Ride capacity increased to {self.capacity}")

# # Mesa Guest Agent with Patience Levels
# Guest Agent (Moves on Grid Toward a Ride)
class GuestAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__( model)  # Correctly initialize the Agent
        self.unique_id = unique_id
        self.destination = None
        self.attraction = None

    def step(self):
        if not self.destination:
            # Ensure rides list is not empty before selecting
            if not self.model.rides:
                return

            # Weighted ride selection: 1/popularity_rank
            ride_weights = [1 / ride.popularity_rank for ride in self.model.rides]
            self.attraction = random.choices(self.model.rides, weights=ride_weights, k=1)[0]
            self.destination = self.attraction.pos

        # Move toward the destination (basic movement)
        x, y = self.pos
        dx, dy = self.destination
        new_x = x + (1 if dx > x else -1 if dx < x else 0)
        new_y = y + (1 if dy > y else -1 if dy < y else 0)

        # Ensure movement stays within grid bounds
        new_x = max(0, min(new_x, self.model.grid.width - 1))
        new_y = max(0, min(new_y, self.model.grid.height - 1))

        self.model.grid.move_agent(self, (new_x, new_y))

        # Arrived at ride
        if self.pos == self.destination:
            print(f"Guest {self.unique_id} arrived at {self.attraction.name}")

# Ride Agent (Fixed in Place)
class RideAgent(Agent):
    def __init__(self, unique_id, model, name, pos,capacity,service_time, popularity_rank):
        super().__init__(model)
        self.unique_id = unique_id
        self.name = name
        self.pos = pos
        self.capacity = capacity
        self.service_time = service_time
        self.popularity_rank = popularity_rank  # Lower is more popular

    def ride_experience(self, guest):
        with self.queue.request() as request:
            queue_length = len(self.queue.queue)
            self.queue_lengths.append(queue_length)  # Track queue length

            yield request  # Wait for turn
            wait_time = self.env.now - guest.arrival_time
            self.wait_times.append(wait_time)

            yield self.env.timeout(self.service_time)  # Ride duration
            print(f"Guest {guest.unique_id} waited {wait_time:.2f} mins and finished at {self.env.now:.2f}")

    def adjust_capacity(self):
        """Dynamically adjust ride capacity if demand is high"""
        if len(self.queue.queue) > 5:  # Increase capacity if long queue
            self.capacity += 1
            self.queue = simpy.Resource(self.env, capacity=self.capacity)
            print(f"Ride capacity increased to {self.capacity}")

# Theme Park Model (Grid-Based)
class ThemeParkGridModel(Model):
    def __init__(self, width, height, num_guests):
        super().__init__()  # Correctly initialize the Model
        self.grid = MultiGrid(width, height, True)  # Initialize grid first
        self.schedule = RandomActivation(self)
        self.randomizer = random  # Use built-in Python random

        # Add rides with popularity ranking
        self.rides = [
            RideAgent(1, self, "Roller Coaster", (5, 5),5,15, 1),
            RideAgent(2, self, "Ferris Wheel", (2, 8),5,15, 3),
            RideAgent(3, self, "Haunted House", (8, 3),5,15, 2),
            RideAgent(4, self, "Bumper Cars", (6, 6),5,15, 4),
        ]
        for ride in self.rides:
            self.schedule.add(ride)
            self.grid.place_agent(ride, ride.pos)

        # Add guests at random positions
        for i in range(num_guests):
            guest = GuestAgent(i + 10, self)
            self.schedule.add(guest)
            x, y = self.randomizer.randint(0, width - 1), self.randomizer.randint(0, height - 1)

            # Ensure guest doesn't spawn on a ride
            while any(ride.pos == (x, y) for ride in self.rides):
                x, y = self.randomizer.randint(0, width - 1), self.randomizer.randint(0, height - 1)

            self.grid.place_agent(guest, (x, y))

    def step(self):
        self.schedule.step()

# Run Simulation
model = ThemeParkGridModel(width=10, height=10, num_guests=10)
for i in range(10):
    model.step()





In [5]:
#Read in data
dir_path = r'C:\Users\User\Downloads\DSA3101_Dataset'  # or unix / linux / mac path
os.chdir(dir_path)
# Get the files from the path provided in the OP
dataframes = {}

for file in os.listdir():
    if file.endswith('.csv'):
        name = file.replace('.csv', '')  # Get the base name without .csv
        dataframes[name] = pd.read_csv(file)  # Store the dataframe with its name as the key



In [35]:
#Reading and editting the dataframes

dataframes
attendance = dataframes.get('attendance')
waiting_times = dataframes.get('waiting_times')
waiting_times
ranking =  waiting_times.groupby('ENTITY_DESCRIPTION_SHORT').agg({
    'GUEST_CARRIED': 'sum',
    'CAPACITY': 'mean'
}).sort_values('GUEST_CARRIED', ascending=False)
ranking

link_attraction_park = dataframes.get('link_attraction_park')
link_attraction_park[['Attraction', 'Park']] = link_attraction_park['ATTRACTION;PARK'].str.split(';', n=1, expand=True)
link_attraction_park


,ATTRACTION;PARK,Attraction,Park
0,Aeroplane Ride;Tivoli Gardens,Aeroplane Ride,Tivoli Gardens
1,Bumper Cars;PortAventura World,Bumper Cars,PortAventura World
2,Bungee Jump;PortAventura World,Bungee Jump,PortAventura World
3,Circus Train;PortAventura World,Circus Train,PortAventura World
4,Crazy Bus;Tivoli Gardens,Crazy Bus,Tivoli Gardens
5,Crazy Dance;PortAventura World,Crazy Dance,PortAventura World
6,Dizzy Dropper;PortAventura World,Dizzy Dropper,PortAventura World
7,Drop Tower;PortAventura World,Drop Tower,PortAventura World
8,Flying Coaster;PortAventura World,Flying Coaster,PortAventura World
9,Free Fall;PortAventura World,Free Fall,PortAventura World


In [36]:
attraction_ranking = ranking.join(link_attraction_park.set_index('Attraction'), on='ENTITY_DESCRIPTION_SHORT')
tivoli_attr_ranking = attraction_ranking[(attraction_ranking['Park'] == 'Tivoli Gardens')]
tivoli_attr_ranking['Ranking'] = range(1, len(tivoli_attr_ranking) + 1)
#tivoli_attr_ranking = tivoli_attr_ranking[['Park', 'Ranking']]
PortAventura_attr_ranking = attraction_ranking[(attraction_ranking['Park'] == 'PortAventura World')]
PortAventura_attr_ranking['Ranking'] = range(1, len(PortAventura_attr_ranking) + 1)
#PortAventura_attr_ranking = PortAventura_attr_ranking[['Park', 'Ranking']]

C:\Users\User\AppData\Local\Temp\ipykernel_12088\1964251836.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tivoli_attr_ranking['Ranking'] = range(1, len(tivoli_attr_ranking) + 1)
C:\Users\User\AppData\Local\Temp\ipykernel_12088\1964251836.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PortAventura_attr_ranking['Ranking'] = range(1, len(PortAventura_attr_ranking) + 1)


In [ ]:
#Inserting data get the average
park1 = simpy.Environment()
tivoli_park = ThemeParkModel(park1)
for _, row in tivoli_attr_ranking.iterrows():
    # Assuming Ride requires (env, capacity, duration, rank)
    attraction = Ride(park1, row['CAPACITY'], 15, row['Ranking'])
    tivoli_park.add_ride(attraction)

park2 = simpy.Environment()
PortAventura = ThemeParkModel(park2)
for _, row in PortAventura_attr_ranking.iterrows():
    # Assuming Ride requires (env, capacity, duration, rank)
    attraction = Ride(park1, row['CAPACITY'], 15, row['Ranking'])
    PortAventura.add_ride(attraction)

